In [2]:
from gensim.models import Word2Vec
import annoy # fast approximate KNN
import numpy as np

In [2]:
!head ./UP000005640.goByGeneC

"V1" "V2"
"A0PJZ0" "MKLFGFRSRRGQTVLGSIDHLYTGSGYRIRYSELQKIHKAAVKGDAAEMERCLARRSGDLDALDKQHRTALHLACASGHVKVVTLLVNRKCQIDIYDKENRTPLIQAVHCQEEACAVILLEHGANPNLKDIYGNTALHYAVYSESTSLAEKLLFHGENIEALDKV" ""
"P30461" "MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRFITVGYVDDTQFVRFDSDATSPRMAPRAPWIEQEGPEYWDRETQISKTNTQTYRENLRTALRYYNQSEAGSHTWQTMYGCDLGPDGRLLRGHNQLAYDGKDYIALNEDLSSWTAADTAAQITQLKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA" ""
"P18825" "MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFF

In [4]:
good_labels = set(['GO:0005634',# nucleus
'GO:0005576',# extracellular
'GO:0005829',# cytosol
'GO:0005856',# cytoskeleton
'GO:0005739',# mitochondrion
'GO:0005886',# cell_membrane
'GO:0005783',# endoplasmic_reticulum
'GO:0005794',# golgi_apparatus
'GO:0005773',# vacuole
'GO:0005777',# peroxysome
              ])

In [5]:
X = [] # sequence strings
Y = [] # list of labels for each sequence
all_y = set() # all the labels collected from the dataset
with open('./UP000005640.goByGeneC') as inp:
    inp.readline() # header
    for line in inp:
        tokens = map(lambda x: x.strip('"'), line.split(' ',2))
        
        object_labels = [t.strip().strip('"') for t in tokens[2].split(',') if len(t.strip())>0 and t.strip]
        object_labels = [y for y in object_labels if y in good_labels]
        if len(object_labels)==1:
            X.append(tokens[1])
            Y.append(object_labels)
            all_y = all_y.union(set(Y[-1]))

In [6]:
print 'example of X', X[:3]
print 'example of Y', Y[:3]
print 'example of set of labels', list(all_y)[:3]
print 'total sequences', len(X)
print 'total labels', len(all_y)

example of X ['MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFFWIGYCNSSLNPVIYTVFNQDFRRSFKHILFRRRRRGFRQ', 'MTRALCSALRQALLLLAAAAELSPGLKCVCLLCDSSNFTCQTEGACWASVMLTNGKEQVIKSCVSLPELNAQVFCHSSNNVTKTECCFTDFCNNITLHLPTASPNAPKLGPMELAIIITVPVCLLSIAAMLTVWACQGRQCSYRKKKRPNVEEPLSECNLVNAGKTLKDLIYDVTASGSGSGLPLLVQRTIARTIVLQEIVGKGRFGEVWHGRWCGEDVAVKIFSSRDERSWFREAEIYQTVMLRHENILGFIAADNKDNGTWTQLWLVSEYHEQGSLYDYLNRNIVTVAGMIKLALSIASGLAHLHMEIVGTQGKPAIAHRDIKSKNILVKKCETCAIADLGLAVKHDSILNTIDIPQNPKVGTKRYMAPEMLDDTMNVNIFESFKRADIYSVGLVYWEIARRCSVGGIVEEYQLPYYDMVPSDPSIEEMRKVVCDQKFRPSIPNQWQSCEALRVMGRIMRECWYANGAARLTALRIKKTISQLCVKEDCKA', 'MAHRKLESVGSGMLDHRVRPGP

In [10]:
def cv_iter(w2v_params):    
    """
    one ieration of cross-validation
    w2v_params is a dictionary with parameters of gensim.Word2Vec
    """
    indices = range(len(X))
    np.random.shuffle(indices)

    Xtrain = np.array(X)[indices[:len(indices)/2]]
    Ytrain = np.array(Y)[indices[:len(indices)/2]]
    Xtest = np.array(X)[indices[len(indices)/2:]]
    Ytest = np.array(Y)[indices[len(indices)/2:]]
    # split all the strings into lists for word2vec traing
    Xtrain = [list(x) for x in Xtrain]
    Xtest = [list(x) for x in Xtest]
    model = Word2Vec(Xtrain, **w2v_params)
    index = annoy.AnnoyIndex(w2v_params.get('size', 100)) # default value is 100 for gensim.Word2Vec
    for i in range(len(Xtrain)):
        v= np.mean([model[c] for c in Xtrain[i] if c in model], axis=0) # centroid
        index.add_item(i, v)
    index.build(100) # 100 is a number of trees in index, we can alibrate this value
    predicted = []
    for i in range(len(Xtest)):
        v= np.mean([model[c] for c in Xtest[i] if c in model], axis=0)
        id = index.get_nns_by_vector(v,1)[0] #  finding 1 nearest neighbour
        predicted.append(id)
    tp = 0
    fp = 0
    true_label_num = 0
    for i in xrange(len(Xtest)):
        true = set(Ytest[i])
        true_label_num+=len(true)
        pred = set(Ytrain[predicted[i]])
        tp += len(true&pred)
        fp += len(pred) - len(true&pred)

    prec=tp*1.0/(tp+fp)
    rec = tp*1.0/true_label_num
    return prec, rec,  2 * prec * rec/ (prec+rec)
    

In [8]:
def cv(w2v_params = {}, cv_num = 5 ):
    """
    Cross-validation function.
    Returns list with precision, recall and F1 values
    """
    precs, recs, fs = [],[],[]
    for _ in range(cv_num):
        p,r,f = cv_iter(w2v_params)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    return precs, recs, fs

In [11]:
p,r,f = cv()
np.mean(p), np.mean(r), np.mean(f)

(0.500738420228239, 0.500738420228239, 0.500738420228239)

In [12]:
p,r,f = cv(w2v_params={'size':5})
np.mean(p), np.mean(r), np.mean(f)

(0.44036697247706413, 0.44036697247706413, 0.44036697247706424)

In [36]:
def cv_iter(w2v_params, classifier):    
    """
    one ieration of cross-validation
    w2v_params is a dictionary with parameters of gensim.Word2Vec
    """
    indices = range(len(X))
    np.random.shuffle(indices)

    Xtrain = np.array(X)[indices[:len(indices)/2]]
    Ytrain = np.array(Y)[indices[:len(indices)/2]]
    Xtest = np.array(X)[indices[len(indices)/2:]]
    Ytest = np.array(Y)[indices[len(indices)/2:]]
    # split all the strings into lists for word2vec traing
    Xtrain = [list(x) for x in Xtrain]
    Xtest = [list(x) for x in Xtest]
    model = Word2Vec(Xtrain, **w2v_params)
    Xtrain2 = []    
    for i in range(len(Xtrain)):
        v= np.mean([model[c] for c in Xtrain[i] if c in model], axis=0) # centroid
        Xtrain2.append(v)
    Xtrain2 = np.array(Xtrain2)
    
    Xtest2 = []    
    for i in range(len(Xtest)):
        v= np.mean([model[c] for c in Xtest[i] if c in model], axis=0) # centroid
        Xtest2.append(v)
    Xtest2 = np.array(Xtest2)
    
    good_labels_idx = list(good_labels)
    Ytrain2 = np.array([good_labels_idx.index(str(y[0])) for y in Ytrain])
    Ytest2 = np.array([good_labels_idx.index(str(y[0])) for y in Ytest])
    cl = classifier()
    print 'fit'
    cl.fit(Xtrain2, Ytrain2)
    print 'ready'
    predicted = cl.predict(Xtest2)
    print predicted[:10]
    print Ytest2[:10]
    tp = 0
    fp = 0
    true_label_num = 0
    for i in xrange(len(Xtest2)):        
        true_label_num+=1
        pred = (Ytest2[i] ==  predicted[i])
        if pred:
            tp+=1
        else:
            fp+=1
        
    prec=tp*1.0/(tp+fp)
    rec = tp*1.0/true_label_num
    return prec, rec,  2 * prec * rec/ (prec+rec)

def cv(w2v_params = {}, cv_num = 5, classifier=None ):
    """
    Cross-validation function.
    Returns list with precision, recall and F1 values
    """
    precs, recs, fs = [],[],[]
    for _ in range(cv_num):
        p,r,f = cv_iter(w2v_params, classifier)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    return precs, recs, fs

In [29]:
from sklearn.linear_model import LogisticRegression
p,r,f = cv(w2v_params={'size':5}, classifier=LogisticRegression)
np.mean(p), np.mean(r), np.mean(f)

(0.5204743790557173, 0.5204743790557173, 0.5204743790557173)

In [30]:
from sklearn.tree import DecisionTreeClassifier
p,r,f = cv(w2v_params={'size':5}, classifier=DecisionTreeClassifier)
np.mean(p), np.mean(r), np.mean(f)

(0.4263146117699709, 0.4263146117699709, 0.4263146117699709)

In [31]:
from sklearn.ensemble import RandomForestClassifier
p,r,f = cv(w2v_params={'size':5}, classifier=RandomForestClassifier)
np.mean(p), np.mean(r), np.mean(f)

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(0.5077646005817856, 0.5077646005817856, 0.5077646005817856)

In [32]:
from sklearn.ensemble import RandomForestClassifier
p,r,f = cv( classifier=RandomForestClassifier)
np.mean(p), np.mean(r), np.mean(f)

(0.5583351980308795, 0.5583351980308795, 0.5583351980308795)

In [39]:
from sklearn.svm import SVC
p,r,f = cv(w2v_params={'size':5}, classifier=SVC)
np.mean(p), np.mean(r), np.mean(f)

fit
ready
[8 8 4 4 4 8 8 8 4 8]
[9 8 2 7 4 9 9 2 8 8]
fit
ready
[8 4 4 4 8 4 4 8 8 8]
[2 4 8 9 8 4 0 7 8 8]
fit
ready
[4 4 4 4 4 4 4 4 8 4]
[9 8 4 9 7 4 1 6 7 9]
fit
ready
[4 4 4 8 4 4 4 4 4 4]
[4 0 6 8 0 4 7 4 9 7]
fit
ready
[8 8 4 8 4 8 4 4 4 8]
[8 1 4 4 4 2 6 4 8 9]


(0.5325576191541732, 0.5325576191541732, 0.5325576191541732)

In [41]:
from sklearn.svm import SVC
p,r,f = cv(classifier=SVC)
np.mean(p), np.mean(r), np.mean(f)

fit
ready
[4 4 4 4 4 4 4 8 4 4]
[7 6 2 4 4 4 7 8 6 9]
fit
ready
[8 4 4 4 8 8 4 4 4 8]
[8 9 8 8 6 8 1 4 4 1]
fit
ready
[4 4 4 8 4 4 4 4 4 4]
[4 4 4 7 8 4 4 6 8 1]
fit
ready
[4 4 8 8 4 4 4 4 4 8]
[6 4 8 8 8 4 8 4 1 8]
fit
ready
[8 4 4 4 8 4 4 4 4 4]
[3 8 4 0 8 4 4 4 4 4]


(0.4983665249496531, 0.4983665249496531, 0.4983665249496531)

In [47]:
from collections import Counter
Counter([y[0] for y in Y])

Counter({'GO:0005576': 1161,
         'GO:0005634': 3224,
         'GO:0005739': 600,
         'GO:0005773': 53,
         'GO:0005777': 18,
         'GO:0005783': 416,
         'GO:0005794': 235,
         'GO:0005829': 677,
         'GO:0005856': 363,
         'GO:0005886': 2191})